In [1]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import re

# 퀴즈1 : 

YES24 국내도서 종합 베스트셀러 정보 특정 페이지 

```
아래의 주소를 참조하여 YES24 종합 베스트셀러의 데이타를 데이타프레임으로 가공하여  csv 파일로 저장하여라. 
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1

저장되는 요소는? 순위, 책제목, 저자, 출판사, 발행일, 가격, 할인가, URL

```


In [3]:
def get_soup(url, headers=None):
    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'lxml')
    return soup

In [4]:
soup = get_soup('http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=4')

In [8]:
def parse_page(url):
    soup = get_soup(url)
    bookid =[a.text for a in soup.find_all('td', class_ = 'num')]
    bookname = [a.find('p').text.replace("\r\n", "").replace("\n", "",) for a in soup.find_all('td', class_ = 'goodsTxtInfo')]
    
    try:
        author  = [re.match('.+?(?= 저| 글)', a.text.replace('\r\n', "").replace('\n', "")).group() for a in soup.find_all('div', class_ = 'aupu')]
    except:
        author = [a.find('a').text for a in soup.find_all('div', class_ = 'aupu')]
        
    publisher= [a.find_all('a')[-1].text for a in soup.find_all('div', class_ = 'aupu')]
    date = [a.text.replace('\r\n', "")[-38:-28] for a in soup.find_all('div', class_ = 'aupu')]
    price = [re.sub('[^0-9]', '', a.find_all('p')[1].text[:8])+"원" for a in soup.find_all('td', class_='goodsTxtInfo')]
    sale_price = [a.find_all('p')[1].find_all('span')[0].text for a in soup.find_all('td', class_='goodsTxtInfo')]
    urls = ["http://www.yes24.com/"+ a.find('a')['href'] for a in soup.find_all('td', class_='goodsTxtInfo')]
    
    data = {"순위":bookid,
       "책이름":bookname,
       "저자":author,
       "출판사":publisher,
       "출판일":date,
       "가격":price,
       "할인가":sale_price,
       "url":urls}
    return data

In [9]:
 pd.DataFrame(parse_page('http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1'))

,순위,책이름,저자,출판사,출판일,가격,할인가,url
0,1.,[도서] 투자는 디테일에 있다 : 슈퍼개미 김정환의 투자 바이블 ...,김정환,부케이,2021년 12월,18000원,"16,200원",http://www.yes24.com//Product/Goods/105261388
1,2.,[도서] 트렌드 코리아 2022 : 서울대 소비트렌드분석센터의 2022 전망(어...,"김난도,전미영,최지혜,이향은,이준영,이수진,서유현,권정윤,한다혜,서울대학교 생활과학...",미래의창,2021년 10월,18000원,"16,200원",http://www.yes24.com//Product/Goods/103737985
2,3.,[도서] 거인의 포트폴리오 : 월급을 쪼개서 경제적 자유를 만드는 23가지 전략...,강환국,페이지2,2021년 11월,18000원,"16,200원",http://www.yes24.com//Product/Goods/104988500
3,4.,[도서] 사랑해 사랑해 사랑해 : 출간 15주년 기념판(보드북) ...,버나뎃 로제티 슈스탁,보물창고,2021년 12월,11800원,"11,210원",http://www.yes24.com//Product/Goods/105163956
4,5.,[도서] 거꾸로 읽는 세계사(전면개정판),유시민,돌베개,2021년 10월,17500원,"15,750원",http://www.yes24.com//Product/Goods/104409067
5,6.,[도서] 흔한남매 겨울밤 대소동 : 흔한남매 스페셜 에디션 : 겨울(크리스마스트...,흔한남매 원저/한바리,미래엔아이세움,2021년 11월,17000원,"15,300원",http://www.yes24.com//Product/Goods/105117824
6,7.,[도서] 달러구트 꿈 백화점 2 : 단골손님을 찾습니다 ...,이미예,팩토리나인,2021년 07월,13800원,"12,420원",http://www.yes24.com//Product/Goods/102789938
7,8.,[도서] 그냥 하지 말라 : 당신의 모든 것이 메시지다(핸드 크림 증정 (색상 ...,송길영,북스톤,2021년 10월,17000원,"15,300원",http://www.yes24.com//Product/Goods/103841300
8,9.,"[도서] Go Go 카카오프렌즈 22 페루(부록 : 캐릭터 스티커 (랩핑), 컬...",김미영,아울북,2021년 11월,12000원,"10,800원",http://www.yes24.com//Product/Goods/105260319
9,10.,[도서] 미드나잇 라이브러리 : 인생의 두 번째 기회를 드립니다(사은품 : 홀로...,매트 헤이그,인플루엔셜,2021년 04월,15800원,"14,220원",http://www.yes24.com//Product/Goods/99534783


# 퀴즈2 : 

YES24 국내도서 종합 베스트셀러 정보 여러 페이지 

```
아래의 주소를 참조하여 YES24 종합 베스트셀러의 특정 페이지(예:1~5페이지)를 범위로 지정하여 
데이타프레임으로 출력되도록한다. 

book_print(시작페이지, 마지막페이지) 형태로 함수를 호출

참조 URL : 
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1

저장되는 요소는? 순위, 책제목, 저자, 출판사, 발행일, 가격, 할인가, URL

```


In [13]:
from tqdm import tqdm

In [11]:
def book_print(startpage, maxpage):
    
    base_url = "http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber={}"
    complete = pd.DataFrame()
    
    for a in tqdm(range(startpage, maxpage+1)):
        url = base_url.format(a)
        temp_df = pd.DataFrame(parse_page(url))
        complete = pd.concat([complete, temp_df])
    return complete
        

In [14]:
# 함수 호출 
a = book_print(1, 50)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.13it/s]


In [15]:
a

,순위,책이름,저자,출판사,출판일,가격,할인가,url
0,1.,[도서] 투자는 디테일에 있다 : 슈퍼개미 김정환의 투자 바이블 ...,김정환,부케이,2021년 12월,18000원,"16,200원",http://www.yes24.com//Product/Goods/105261388
1,2.,[도서] 트렌드 코리아 2022 : 서울대 소비트렌드분석센터의 2022 전망(어...,"김난도,전미영,최지혜,이향은,이준영,이수진,서유현,권정윤,한다혜,서울대학교 생활과학...",미래의창,2021년 10월,18000원,"16,200원",http://www.yes24.com//Product/Goods/103737985
2,3.,[도서] 거인의 포트폴리오 : 월급을 쪼개서 경제적 자유를 만드는 23가지 전략...,강환국,페이지2,2021년 11월,18000원,"16,200원",http://www.yes24.com//Product/Goods/104988500
3,4.,[도서] 사랑해 사랑해 사랑해 : 출간 15주년 기념판(보드북) ...,버나뎃 로제티 슈스탁,보물창고,2021년 12월,11800원,"11,210원",http://www.yes24.com//Product/Goods/105163956
4,5.,[도서] 거꾸로 읽는 세계사(전면개정판),유시민,돌베개,2021년 10월,17500원,"15,750원",http://www.yes24.com//Product/Goods/104409067
...,...,...,...,...,...,...,...,...
14,995.,[도서] 슬픈 세상의 기쁜 말 : 당신을 살아 있게 하는 말은 무엇입니까(요시고...,정혜윤,위고,2021년 08월,16000원,"14,400원",http://www.yes24.com//Product/Goods/103028898
15,996.,[도서] 메타버스의 시대 : 미래의 부와 기회를 선점하는 7대 메가트렌드 ...,이시한,다산북스,2021년 08월,17000원,"15,300원",http://www.yes24.com//Product/Goods/103302716
16,997.,[도서] 절대로 누르면 안 돼! 크리스마스에도(초판 한정 부록 : 크리스마스 엽...,빌 코터,북뱅크,2020년 12월,12000원,"10,800원",http://www.yes24.com//Product/Goods/95149669
17,998.,[도서] 이상하게 돈 걱정 없는 사람들의 비밀,혼다 고이치,동양북스(동양books),2021년 11월,13800원,"12,420원",http://www.yes24.com//Product/Goods/104801308


# 퀴즈3  

YES24 국내도서 종합 베스트셀러 정보 모든 페이지 

```
아래의 주소를 참조하여 YES24 종합 베스트셀러의 모든 페이지가 
데이타프레임으로 출력되고 csv로 저장되도록한다. 

참조 URL : 
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1

저장되는 요소는? 순위, 책제목, 저자, 출판사, 발행일, 가격, 할인가, URL

```



In [62]:
def book_print2():
    
    base_url = "http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber={}"
    page = 0
    complete = pd.DataFrame()
    
    while True:
        page += 1
        url = base_url.format(page)
        print(url)
        temp_df = pd.DataFrame(parse_page(url))
    complete = pd.concat([complete, temp])
    return temp ,complete
        

In [17]:
# 함수 호출 
c = book_print(1,51)


100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:22<00:00,  2.30it/s]


In [18]:
c

,순위,책이름,저자,출판사,출판일,가격,할인가,url
0,1.,[도서] 투자는 디테일에 있다 : 슈퍼개미 김정환의 투자 바이블 ...,김정환,부케이,2021년 12월,18000원,"16,200원",http://www.yes24.com//Product/Goods/105261388
1,2.,[도서] 트렌드 코리아 2022 : 서울대 소비트렌드분석센터의 2022 전망(어...,"김난도,전미영,최지혜,이향은,이준영,이수진,서유현,권정윤,한다혜,서울대학교 생활과학...",미래의창,2021년 10월,18000원,"16,200원",http://www.yes24.com//Product/Goods/103737985
2,3.,[도서] 거인의 포트폴리오 : 월급을 쪼개서 경제적 자유를 만드는 23가지 전략...,강환국,페이지2,2021년 11월,18000원,"16,200원",http://www.yes24.com//Product/Goods/104988500
3,4.,[도서] 사랑해 사랑해 사랑해 : 출간 15주년 기념판(보드북) ...,버나뎃 로제티 슈스탁,보물창고,2021년 12월,11800원,"11,210원",http://www.yes24.com//Product/Goods/105163956
4,5.,[도서] 거꾸로 읽는 세계사(전면개정판),유시민,돌베개,2021년 10월,17500원,"15,750원",http://www.yes24.com//Product/Goods/104409067
...,...,...,...,...,...,...,...,...
14,995.,[도서] 슬픈 세상의 기쁜 말 : 당신을 살아 있게 하는 말은 무엇입니까(요시고...,정혜윤,위고,2021년 08월,16000원,"14,400원",http://www.yes24.com//Product/Goods/103028898
15,996.,[도서] 메타버스의 시대 : 미래의 부와 기회를 선점하는 7대 메가트렌드 ...,이시한,다산북스,2021년 08월,17000원,"15,300원",http://www.yes24.com//Product/Goods/103302716
16,997.,[도서] 절대로 누르면 안 돼! 크리스마스에도(초판 한정 부록 : 크리스마스 엽...,빌 코터,북뱅크,2020년 12월,12000원,"10,800원",http://www.yes24.com//Product/Goods/95149669
17,998.,[도서] 이상하게 돈 걱정 없는 사람들의 비밀,혼다 고이치,동양북스(동양books),2021년 11월,13800원,"12,420원",http://www.yes24.com//Product/Goods/104801308


In [67]:
c.shape

(999, 8)

In [21]:
c.to_csv('yes24bestseller_202111_29.csv', index=False, encoding= 'utf-8-sig')

In [ ]:
a